In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 1
setup_one_gpu(GPU)

Picking GPU 1


In [2]:
import sys
import numpy as np
import os.path as osp
from evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from helper import stored_synthetic_samples, top_evaluation_dir

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
class_name = 'chair'

do_jsd = False
do_mmd = True
save_res = False

top_out_dir = top_evaluation_dir
experiment_name = 'l_gan_emd'

sample_mmd = True
n_samples = 5
ref_pop = 1000
sample_pop = 1000
mmd_loss = 'chamfer'
special_tag = 'emd_40_200'

# skip = ['test', 'val']
skip = ['train', 'test']

In [5]:
if do_jsd:
    boost_samples = 1
else:
    boost_samples = 5
assert(np.logical_xor(do_jsd, do_mmd))

In [6]:
top_sample_dir = stored_synthetic_samples(class_name)[experiment_name]

In [7]:
if save_res:
    exp_name = osp.basename(top_sample_dir)
    if do_jsd:
        f_jsd = open(osp.join(top_out_dir, class_name, 'jsd_' + special_tag + exp_name + '.txt'), 'w')
            
    if do_mmd:
        f_mmd = open(osp.join(top_out_dir, class_name, 'mmd_' + special_tag + exp_name + '.txt'), 'w')
else:
    f_jsd = sys.stdout
    f_mmd = sys.stdout

In [8]:
if experiment_name.startswith('l_') and 'gan' in experiment_name:
    epochs_to_check = np.hstack([np.array([1, 5, 10]), np.arange(100, 501, 100)])
elif experiment_name.startswith('r_gan'):
    epochs_to_check = np.hstack([np.array([1, 5, 10]), np.arange(50, 2001, 50)])

In [9]:
epochs_to_check = [1700]

In [10]:
sample_files = []
if 'gan' in experiment_name:
    for epoch in epochs_to_check:
        sample_files.append(osp.join(top_sample_dir, 'epoch_%d.npz' % (epoch,) ))
elif 'gmm' in experiment_name:
    for f in files_in_subdirs(top_sample_dir, '.npz'):
        sample_files.append(f) 
else:
    assert False

In [11]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()
if do_jsd:
    evaluator.prepare_gt_grid_variables()

In [12]:
for sample_file in sample_files:
    evaluator.prepare_sample_data(sample_file, boost_samples)
    sample_name = osp.basename(sample_file)[:-len('.npz')]
    print sample_name
    if do_jsd:
        f_jsd.write(sample_name + '\n')
        evaluator.compute_jsd(f_out=f_jsd)
        f_jsd.flush()
    if do_mmd:
        f_mmd.write(sample_name + '\n')
        evaluator.compute_mmd(loss=mmd_loss, sample_estimator=sample_mmd, n_samples=n_samples, 
                              ref_pop_size=ref_pop, sample_pop_size=sample_pop,
                              f_out=f_mmd, skip=skip)
        
        f_mmd.flush()
    print

epoch_1700
epoch_1700
val 0.00188163 0.00102614



In [ ]:
if save_res:
    if do_jsd:
        f_jsd.close()
    if do_mmd:
        f_mmd.close()